<a href="https://colab.research.google.com/github/MarvinLopezOsorio/AnaliticaPredictiva_TrabajoFinal/blob/main/Trabajo%20Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

In [2]:
myfile = pd.read_excel('https://github.com/MarvinLopezOsorio/AnaliticaPredictiva_TrabajoFinal/blob/main/Base%20de%20Datos%20Universidad.xlsx?raw=true')

,Ingreso,Estado,Sexo,Municipio,Edad,Paes,Carrera,Facultad,Segunda carrera,CUM,Porcentaje de avance,Estado actual,Razonamiento Verbal (DAT- VR),Relaciones Espaciales (DAT - SR),Razonamiento Abstracto (DAT- AR),Aptitud Numérica (DAT - NA),Razonamiento Mecánico (DAT-MR),Rapidez y precisión perceptiva (DAT -CSA),Hábitos,Aprobación,tipo institución
0,2017,ACTIVO,Femenino,Soyapango,23,5.52,Licenciatura en Idiomas con especialidad en la...,Facultad de CC. y Humanidades,NaN,8.4,77.14,ACTIVO,75,15,20,5,45,5,5,Aprobado,Privado
1,2017,ACTIVO,Femenino,Soyapango,23,4.50,Licenciatura en Ciencias de la Comunicación,Facultad de CC. y Humanidades,NaN,8.6,94.00,ACTIVO,75,20,15,50,15,5,1,Aprobado,Privado
2,2017,ACTIVO,Femenino,Soyapango,24,3.83,Licenciatura en Idiomas con especialidad en Tu...,Facultad de CC. y Humanidades,NaN,7.6,79.41,ACTIVO,45,5,10,30,15,5,4,Aprobado,Público
3,2017,ACTIVO,Femenino,Soyapango,23,7.85,Licenciatura en Diseño Industrial y de Productos,Facultad de CC. y Humanidades,NaN,7.8,85.60,ACTIVO,90,100,95,60,65,5,7,Aprobado,Público
4,2017,ACTIVO,Femenino,Soyapango,23,5.14,Licenciatura en Diseño Gráfico,Facultad de CC. y Humanidades,NaN,8.2,95.20,ACTIVO,25,60,10,50,95,5,2,Aprobado,Privado
